In [ ]:
import pandas as pd

# Define the file path
file_path = r"/content/collab.xlsx"

# --- 0. Define Lookup Lists for Accurate Parsing ---

# List of all industries
INDUSTRY_NAMES = {
    "Agriculture, Forestry, and Fishing",
    "Mining and Quarrying",
    "Manufacturing",
    "Electricity, Steam, Water and Waste Management",
    "Construction",
    "Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles",
    "Transportation and Storage",
    "Accommodation and Food Service Activities",
    "Information and Communication",
    "Financial and Insurance Activities",
    "Real Estate and Ownership of Dwellings",
    "Professional and Business Services",
    "Public Administration and Defense; Compulsory Social Activities",
    "Education",
    "Human Health and Social Work Activities",
    "Other Services"
}

# List of all regions
REGION_NAMES = {
    "National Capital Region (NCR)",
    "Cordillera Administrative Region (CAR)",
    "Region I (Ilocos Region)",
    "Region II (Cagayan Valley)",
    "Region III (Central Luzon)",
    "Region IV-A (CALABARZON)",
    "Region IV-B (MIMAROPA)",
    "Region V (Bicol Region)",
    "Region VI (Western Visayas)",
    "Region VII (Central Visayas)",
    "Region VIII (Eastern Visayas)",
    "Region IX (Zamboanga Peninsula)",
    "Region X (Northern Mindanao)",
    "Region XI (Davao Region)",
    "Region XII (SOCCSKSARGEN)",
    "Region XIII (Caraga)",
    "Bangsamoro Autonomous Region in Muslim Mindanao (BARMM)",
}

try:
    print(f"Attempting to load file from: {file_path}")
    # 1. Load the data
    excel_sheets = pd.ExcelFile(file_path).sheet_names
    print(f"Sheets found in '{file_path}': {excel_sheets}")

    # --- ACTION REQUIRED: Confirm/Change this to the correct sheet name if not 'Sheet1' ---
    correct_sheet_name = excel_sheets[0] # Default to the first sheet

    df = pd.read_excel(file_path, sheet_name=correct_sheet_name, header=None)
    print(f"\nSuccessfully loaded data from sheet: '{correct_sheet_name}'")

    # 2. Clean up the Header
    # Corrected fillna to ffill() to address FutureWarning
    header_level_1 = df.iloc[1, :].ffill(axis=0)
    header_level_2 = df.iloc[2, :]

    new_columns = []
    for i in range(len(header_level_1)):
        if i < 2:
            val_0 = df.iloc[0, i]
            val_1 = df.iloc[1, i]
            val_2 = df.iloc[2, i]
            if pd.notna(val_0):
                new_columns.append(val_0)
            elif pd.notna(val_1):
                new_columns.append(val_1)
            else:
                new_columns.append(val_2)
        else:
            metric = header_level_1.iloc[i]
            year_or_range = header_level_2.iloc[i]
            if pd.isna(metric) and pd.isna(year_or_range):
                new_columns.append(f"Unnamed_Col_{i}")
            elif pd.isna(metric):
                new_columns.append(f"Unnamed_Metric_{year_or_range}")
            elif pd.isna(year_or_range):
                new_columns.append(f"{metric}_Unnamed_Year")
            else:
                new_columns.append(f"{metric}_{year_or_range}")

    df.columns = new_columns
    df_data = df.iloc[3:].copy()
    df_data = df_data.rename(columns={df_data.columns[0]: 'Industry_Raw', df_data.columns[1]: 'Location_Raw'})

    # 3. Process Industry and Location Hierarchy
    final_industries = []
    final_regions = []
    final_location_types = []
    final_location_names = []

    current_industry = None
    current_region = None

    for idx, row in df_data.iterrows():
        industry_raw_name = str(row['Industry_Raw']).strip() if pd.notna(row['Industry_Raw']) else None
        location_raw_name = str(row['Location_Raw']).strip() if pd.notna(row['Location_Raw']) else None

        if industry_raw_name in INDUSTRY_NAMES:
            current_industry = industry_raw_name
        final_industries.append(current_industry)

        if location_raw_name in REGION_NAMES:
            current_region = location_raw_name
            final_regions.append(current_region)
            final_location_types.append('Region')
            final_location_names.append(location_raw_name)
        elif location_raw_name is None or location_raw_name == 'nan':
            final_regions.append(None)
            final_location_types.append(None)
            final_location_names.append(None)
        else:
            final_regions.append(current_region)
            if 'City of' in location_raw_name:
                final_location_types.append('City')
            elif 'Province of' in location_raw_name:
                final_location_types.append('Province')
            elif any(prov in location_raw_name for prov in ["Abra", "Apayao", "Benguet", "Ifugao", "Kalinga", "Mountain Province"]):
                final_location_types.append('Province')
            else:
                final_location_types.append('Province/City')
            final_location_names.append(location_raw_name)

    df_data['Industry'] = final_industries
    df_data['Region'] = final_regions
    df_data['Location_Type'] = final_location_types
    df_data['Location_Name'] = final_location_names

    df_data = df_data.drop(columns=['Industry_Raw', 'Location_Raw'])
    df_data.dropna(subset=['Location_Name'], inplace=True)

    # 4. Melt (Unpivot) the Data
    id_vars = ['Industry', 'Region', 'Location_Type', 'Location_Name']
    value_vars = [col for col in df_data.columns if col not in id_vars]

    df_melted = df_data.melt(id_vars=id_vars,
                             value_vars=value_vars,
                             var_name='Metric_Year_Range',
                             value_name='Value')

    # 5. Split the 'Metric_Year_Range' column
    df_melted[['Metric', 'Year_Range']] = df_melted['Metric_Year_Range'].str.rsplit('_', n=1, expand=True)
    df_melted = df_melted.drop(columns=['Metric_Year_Range'])

    # 6. Clean up values and data types - retains NaNs from '/s'
    df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')

    def parse_year_range(year_range_str):
        if '-' in str(year_range_str):
            start, end = map(int, year_range_str.split('-'))
            return int(start), int(end)
        else:
            try:
                year = int(year_range_str)
                return year, year
            except ValueError:
                return None, None

    df_melted[['Start_Year', 'End_Year']] = df_melted['Year_Range'].apply(parse_year_range).apply(pd.Series)

    final_df = df_melted[['Industry', 'Region', 'Location_Type', 'Location_Name',
                           'Metric', 'Year_Range', 'Start_Year', 'End_Year', 'Value']].copy()

    # Ensure consistent data types
    final_df['Industry'] = final_df['Industry'].astype(str)
    final_df['Region'] = final_df['Region'].astype(str)
    final_df['Location_Type'] = final_df['Location_Type'].astype(str)
    final_df['Location_Name'] = final_df['Location_Name'].astype(str)
    final_df['Metric'] = final_df['Metric'].astype(str)
    final_df['Year_Range'] = final_df['Year_Range'].astype(str)
    final_df['Start_Year'] = pd.to_numeric(final_df['Start_Year'], errors='coerce').astype('Int64')
    final_df['End_Year'] = pd.to_numeric(final_df['End_Year'], errors='coerce').astype('Int64')

    # --- Debugging: Inspect available Metrics and Year Ranges ---
    print("\n--- Available Metrics and their counts in the dataset ---")
    print(final_df['Metric'].value_counts().to_markdown())

    print("\n--- Available Year Ranges for GDP ---")
    print(final_df[final_df['Metric'] == 'GDP']['Year_Range'].value_counts().to_markdown())

    print("\n--- Available Year Ranges for Growth Rate ---")
    print(final_df[final_df['Metric'] == 'Growth Rate']['Year_Range'].value_counts().to_markdown())

    print("\n--- Available Year Ranges for Percent Share ---")
    print(final_df[final_df['Metric'] == 'Percent Share']['Year_Range'].value_counts().to_markdown())


    # --- Set Pandas display options to show all rows and columns for comprehensive output ---
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)

    print("\n--- Filtering for GDP Data (2018-2023) ---")
    gdp_2018_2023 = final_df[
        (final_df['Metric'] == 'GDP') &
        (final_df['Start_Year'] >= 2018) &
        (final_df['End_Year'] <= 2023)
    ].sort_values(by=['Industry', 'Region', 'Location_Name', 'Start_Year'])

    if not gdp_2018_2023.empty:
        print(gdp_2018_2023.to_markdown(index=False))
    else:
        print("No GDP data found for the years 2018-2023 based on the current filtering criteria. This could mean either no data exists for these years or the year ranges in the source are different (e.g., '2017-2018' instead of individual years '2018', '2019'). Please check the 'Available Year Ranges for GDP' above.")

    print("\n--- Filtering for Growth Rate Data (2018-2023) ---")
    growth_rate_2018_2023 = final_df[
        (final_df['Metric'] == 'Growth Rate') &
        (final_df['Start_Year'] >= 2018) &
        (final_df['End_Year'] <= 2023)
    ].sort_values(by=['Industry', 'Region', 'Location_Name', 'Start_Year'])

    if not growth_rate_2018_2023.empty:
        print(growth_rate_2018_2023.to_markdown(index=False))
    else:
        print("No Growth Rate data found for the years 2018-2023 based on the current filtering criteria. This could mean either no data exists for these years or the year ranges in the source are different (e.g., '2017-2018' instead of '2018-2019', etc.). Please check the 'Available Year Ranges for Growth Rate' above.")

    print("\n--- Filtering for Percent Share Data (2018-2023) ---")
    percent_share_2018_2023 = final_df[
        (final_df['Metric'] == 'Percent Share') &
        (final_df['Start_Year'] >= 2018) &
        (final_df['End_Year'] <= 2023)
    ].sort_values(by=['Industry', 'Region', 'Location_Name', 'Start_Year'])

    if not percent_share_2018_2023.empty:
        print(percent_share_2018_2023.to_markdown(index=False))
    else:
        print("No Percent Share data found for the years 2018-2023 based on the current filtering criteria. Please check the 'Available Year Ranges for Percent Share' above.")

    # Reset Pandas display options to default
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')

    # Generate a CSV file of the cleaned data
    final_df.to_csv("cleaned_data.csv", index=False)
    print("\nCleaned data has been saved to 'cleaned_data.csv'")

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please ensure it's uploaded to /content/.")
    print("If you just restarted the Colab runtime, you might need to re-upload the file.")
    print("You can upload it using: `from google.colab import files; uploaded = files.upload()`")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Attempting to load file from: /content/collab.xlsx
Error: The file '/content/collab.xlsx' was not found. Please ensure it's uploaded to /content/.
If you just restarted the Colab runtime, you might need to re-upload the file.
You can upload it using: `from google.colab import files; uploaded = files.upload()`


In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the file path to the cleaned Excel file
file_path = r"/content/cleaned_data.xlsx"

try:
    print(f"Attempting to load cleaned data from: {file_path}")
    # Load the cleaned data directly into final_df using pd.read_excel
    final_df = pd.read_excel(file_path) # Changed from pd.read_csv to pd.read_excel
    print(f"\nSuccessfully loaded cleaned data from: '{file_path}'")

    # Ensure correct data types, as reading from Excel might alter them
    final_df['Industry'] = final_df['Industry'].astype(str)
    final_df['Region'] = final_df['Region'].astype(str)
    final_df['Location_Type'] = final_df['Location_Type'].astype(str)
    final_df['Location_Name'] = final_df['Location_Name'].astype(str)
    final_df['Metric'] = final_df['Metric'].astype(str)
    final_df['Year_Range'] = final_df['Year_Range'].astype(str)
    # Use 'Int64' for integer columns that might contain NaN
    final_df['Start_Year'] = pd.to_numeric(final_df['Start_Year'], errors='coerce').astype('Int64')
    final_df['End_Year'] = pd.to_numeric(final_df['End_Year'], errors='coerce').astype('Int64')
    final_df['Value'] = pd.to_numeric(final_df['Value'], errors='coerce')

    # --- DEBUGGING: Inspect available values in critical columns (for my reference) ---
    print("\n--- Value Counts for 'Metric' in cleaned_data.xlsx ---")
    print(final_df['Metric'].value_counts().to_markdown())

    print("\n--- Value Counts for 'Location_Type' in cleaned_data.xlsx ---")
    print(final_df['Location_Type'].value_counts().to_markdown())

    print("\n--- Value Counts for 'Industry' in cleaned_data.xlsx (Top 20) ---")
    print(final_df['Industry'].value_counts().head(20).to_markdown())

    print("\n--- Value Counts for 'Region' in cleaned_data.xlsx (Top 20) ---")
    print(final_df['Region'].value_counts().head(20).to_markdown())


    # --- FORECASTING REGIONAL GDP ---

    # Filter for GDP data at the Region level
    # Aggregate by summing industries to get total regional GDP
    gdp_regional_sectors = final_df[
        (final_df['Metric'] == 'GDP') &
        (final_df['Location_Type'] == 'Region') &
        (final_df['Region'] != 'nan') # Exclude 'nan' regions
    ].copy()

    # Aggregate by summing GDP across all industries for each Region and Year
    gdp_data = gdp_regional_sectors.groupby(['Region', 'Location_Name', 'Start_Year'])['Value'].sum().reset_index()

    # Convert 'Start_Year' to datetime for Prophet's 'ds' column
    gdp_data['ds'] = pd.to_datetime(gdp_data['Start_Year'], format='%Y')
    gdp_data['y'] = gdp_data['Value'] # Rename 'Value' to 'y' for Prophet

    # Drop rows where 'y' is NaN, as Prophet will use historical data for training
    gdp_data.dropna(subset=['y'], inplace=True)

    unique_regions = gdp_data['Region'].unique()
    all_forecasts = pd.DataFrame()

    print(f"\n--- Starting GDP Forecast for {len(unique_regions)} Regions ---")
    if len(unique_regions) == 0:
        raise ValueError("No regions found with sufficient data after aggregation for forecasting.")


    # Define the number of years to forecast into the future
    forecast_years = 5

    for region in unique_regions:
        print(f"Forecasting for: {region}")
        region_df = gdp_data[gdp_data['Region'] == region][['ds', 'y']].sort_values('ds')

        if region_df.empty or len(region_df) < 2: # Prophet needs at least 2 data points
            print(f"Not enough data for {region} to forecast. Skipping.")
            continue

        # Initialize and fit Prophet model
        m = Prophet(yearly_seasonality=True)
        m.fit(region_df)

        # Create future dataframe for forecasting
        future = m.make_future_dataframe(periods=forecast_years, freq='Y')
        forecast = m.predict(future)

        # Merge historical data with forecast for plotting
        forecast['Region'] = region
        merged_df = forecast.merge(region_df.rename(columns={'y': 'Actual'}), on='ds', how='left')
        merged_df = merged_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'Region', 'Actual']]
        all_forecasts = pd.concat([all_forecasts, merged_df], ignore_index=True)

    # --- Create Interactive Plotly Chart ---

    fig = go.Figure()

    total_traces_per_region = 4
    first_region = unique_regions[0] if unique_regions.size > 0 else None
    sorted_unique_regions = sorted(unique_regions.tolist())

    for i, region in enumerate(sorted_unique_regions):
        region_forecasts = all_forecasts[all_forecasts['Region'] == region].copy()

        hist_data_for_plot = region_forecasts[region_forecasts['Actual'].notna()].drop_duplicates(subset=['ds'])
        forecast_data_for_plot = region_forecasts.drop_duplicates(subset=['ds'])

        initial_visibility = True if i == 0 else False

        # 1. Actual GDP (Historical)
        fig.add_trace(go.Scatter(
            x=hist_data_for_plot['ds'],
            y=hist_data_for_plot['Actual'],
            mode='lines+markers',
            name='Actual GDP',
            line=dict(color='blue'),
            visible=initial_visibility,
            showlegend=True
        ))
        # 2. Forecasted GDP Line
        fig.add_trace(go.Scatter(
            x=forecast_data_for_plot['ds'],
            y=forecast_data_for_plot['yhat'],
            mode='lines',
            name='Forecasted GDP',
            line=dict(color='red', dash='dash'),
            visible=initial_visibility,
            showlegend=True
        ))
        # 3. Forecast Upper Confidence Interval
        fig.add_trace(go.Scatter(
            x=forecast_data_for_plot['ds'],
            y=forecast_data_for_plot['yhat_upper'],
            fill=None,
            mode='lines',
            line_color='rgba(255,0,0,0)',
            showlegend=False,
            visible=initial_visibility
        ))
        # 4. Forecast Lower Confidence Interval (filled to upper)
        fig.add_trace(go.Scatter(
            x=forecast_data_for_plot['ds'],
            y=forecast_data_for_plot['yhat_lower'],
            fill='tonexty',
            mode='lines',
            line_color='rgba(255,0,0,0)',
            fillcolor='rgba(255,0,0,0.2)',
            name='Confidence Interval',
            visible=initial_visibility,
            showlegend=True
        ))

    # Create dropdown menu buttons
    buttons = []
    for i, region in enumerate(sorted_unique_regions):
        button_visibility = [False] * (len(sorted_unique_regions) * total_traces_per_region)
        button_visibility[i * total_traces_per_region] = True
        button_visibility[i * total_traces_per_region + 1] = True
        button_visibility[i * total_traces_per_region + 2] = True
        button_visibility[i * total_traces_per_region + 3] = True

        buttons.append(dict(
            label=region,
            method="update",
            args=[{"visible": button_visibility},
                  {"title": f"Regional GDP Forecast for {region}"}]
        ))

    if first_region:
        fig.update_layout(
            updatemenus=[
                go.layout.Updatemenu(
                    type="dropdown",
                    direction="down",
                    active=0,
                    x=0.0,
                    xanchor="left",
                    y=1.02,
                    yanchor="bottom",
                    buttons=buttons,
                    pad={"r": 10, "t": 10}
                )
            ],
            title_text=f"Regional GDP Forecast for {first_region}",
            yaxis_title="GDP Value",
            xaxis_title="Year",
            hovermode="x unified",
            height=600,
            showlegend=True,
            margin=dict(l=50, r=50, t=120, b=50),
            legend=dict(
                x=0.99,
                y=0.99,
                xanchor='right',
                yanchor='top',
                bgcolor='rgba(255,255,255,0.7)',
                bordercolor='rgba(0,0,0,0.2)',
                borderwidth=1
            )
        )
    else:
        fig.update_layout(
            title_text="No sufficient regional GDP data to generate forecasts. Please check the 'Value Counts' above.",
            xaxis_title="Year",
            yaxis_title="GDP Value",
            height=600,
            showlegend=False
        )

    fig.show()

    # Add this line to save the chart as an HTML file
    fig.write_html("Forecasted Regional GDP.html")
    print("\nInteractive chart saved as 'Forecasted Regional GDP.html'")

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please ensure it's uploaded to /content/.")
    print("If you just restarted the Colab runtime, you might need to re-upload the file.")
    print("You can upload it using: `from google.colab import files; uploaded = files.upload()`")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Attempting to load cleaned data from: /content/cleaned_data.xlsx

Successfully loaded cleaned data from: '/content/cleaned_data.xlsx'

--- Value Counts for 'Metric' in cleaned_data.xlsx ---
| Metric        |   count |
|:--------------|--------:|
| GDP           |   12870 |
| Percent Share |   12870 |
| Growth Rate   |   10725 |

--- Value Counts for 'Location_Type' in cleaned_data.xlsx ---
| Location_Type   |   count |
|:----------------|--------:|
| Province/City   |   21505 |
| City            |    8704 |
| Region          |    4624 |
| Province        |    1632 |

--- Value Counts for 'Industry' in cleaned_data.xlsx (Top 20) ---
| Industry                                                             |   count |
|:---------------------------------------------------------------------|--------:|
| Other Services                                                       |    2295 |
| Agriculture, Forestry, and Fishing                                   |    2278 |
| Manufacturing             

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/dkjgws5a.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/qu8zks8u.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=34718', 'data', 'file=/tmp/tmp33v_ubrc/dkjgws5a.json', 'init=/tmp/tmp33v_ubrc/qu8zks8u.json', 'output', 'file=/tmp/tmp33v_ubrc/prophet_modelk5e94dbc/prophet_model-20250705074621.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:46:21 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:46:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:c

Forecasting for: Cordillera Administrative Region (CAR)
Forecasting for: National Capital Region (NCR)


/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/dbozkg7k.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/zpwt_yhr.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=66339', 'data', 'file=/tmp/tmp33v_ubrc/dbozkg7k.json', 'init=/tmp/tmp33v_ubrc/zpwt_yhr.json', 'output', 'file=/tmp/tmp33v_ubrc/prophet_modelq77d08_h/prophet_model-20250705074633.csv', 'method=optimize', 'al

Forecasting for: Region I (Ilocos Region)
Forecasting for: Region II (Cagayan Valley)


...

Forecasting for: Region XII (SOCCSKSARGEN)


07:46:46 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/_iplgegy.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/78vjukhr.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=40534', 'data', 'file=/tmp/tmp33v_ubrc/_iplgegy.json', 'init=/tmp/tmp33v_ubrc/78vjukhr.json', 'output', 'file

Forecasting for: Region XIII (Caraga)


07:46:51 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(



Interactive chart saved as 'Forecasted Regional GDP.html'


In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the file path to the cleaned CSV
file_path = r"/content/cleaned_data.csv"

try:
    print(f"Attempting to load cleaned data from: {file_path}")
    # Load the cleaned data directly into final_df
    final_df = pd.read_csv(file_path)
    print(f"\nSuccessfully loaded cleaned data from: '{file_path}'")

    # Ensure correct data types, as reading from CSV might alter them
    final_df['Industry'] = final_df['Industry'].astype(str)
    final_df['Region'] = final_df['Region'].astype(str)
    final_df['Location_Type'] = final_df['Location_Type'].astype(str)
    final_df['Location_Name'] = final_df['Location_Name'].astype(str)
    final_df['Metric'] = final_df['Metric'].astype(str)
    final_df['Year_Range'] = final_df['Year_Range'].astype(str)
    # Use 'Int64' for integer columns that might contain NaN
    final_df['Start_Year'] = pd.to_numeric(final_df['Start_Year'], errors='coerce').astype('Int64')
    final_df['End_Year'] = pd.to_numeric(final_df['End_Year'], errors='coerce').astype('Int64')
    final_df['Value'] = pd.to_numeric(final_df['Value'], errors='coerce')


    # --- FORECASTING REGIONAL GDP ---

    # Filter for GDP data at the Region level
    gdp_regional_sectors = final_df[
        (final_df['Metric'] == 'GDP') &
        (final_df['Location_Type'] == 'Region') &
        (final_df['Region'] != 'nan') # Exclude 'nan' regions
    ].copy()

    # Aggregate by summing GDP across all industries for each Region and Year
    gdp_data = gdp_regional_sectors.groupby(['Region', 'Location_Name', 'Start_Year'])['Value'].sum().reset_index()

    # Convert 'Start_Year' to datetime for Prophet's 'ds' column
    gdp_data['ds'] = pd.to_datetime(gdp_data['Start_Year'], format='%Y')
    gdp_data['y'] = gdp_data['Value'] # Rename 'Value' to 'y' for Prophet

    # Drop rows where 'y' is NaN, as Prophet will use historical data for training
    gdp_data.dropna(subset=['y'], inplace=True)

    unique_regions = gdp_data['Region'].unique()
    all_forecasts = pd.DataFrame()

    print(f"\n--- Starting GDP Forecast for {len(unique_regions)} Regions ---")
    if len(unique_regions) == 0:
        raise ValueError("No regions found with sufficient data after aggregation for forecasting.")


    # Define the number of years to forecast into the future
    forecast_years = 5

    for region in unique_regions:
        print(f"Forecasting for: {region}")
        region_df = gdp_data[gdp_data['Region'] == region][['ds', 'y']].sort_values('ds')

        if region_df.empty or len(region_df) < 2: # Prophet needs at least 2 data points
            print(f"Not enough data for {region} to forecast. Skipping.")
            continue

        # Initialize and fit Prophet model
        m = Prophet(yearly_seasonality=True)
        m.fit(region_df)

        # Create future dataframe for forecasting
        future = m.make_future_dataframe(periods=forecast_years, freq='Y')
        forecast = m.predict(future)

        # Merge historical data with forecast for plotting
        forecast['Region'] = region
        merged_df = forecast.merge(region_df.rename(columns={'y': 'Actual'}), on='ds', how='left')
        merged_df = merged_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'Region', 'Actual']]
        all_forecasts = pd.concat([all_forecasts, merged_df], ignore_index=True)

    # --- Create Interactive Plotly Chart ---

    fig = go.Figure()

    total_traces_per_region = 4
    first_region = unique_regions[0] if unique_regions.size > 0 else None
    sorted_unique_regions = sorted(unique_regions.tolist())

    for i, region in enumerate(sorted_unique_regions):
        region_forecasts = all_forecasts[all_forecasts['Region'] == region].copy()

        hist_data_for_plot = region_forecasts[region_forecasts['Actual'].notna()].drop_duplicates(subset=['ds'])
        forecast_data_for_plot = region_forecasts.drop_duplicates(subset=['ds'])

        initial_visibility = True if i == 0 else False

        # 1. Actual GDP (Historical)
        fig.add_trace(go.Scatter(
            x=hist_data_for_plot['ds'],
            y=hist_data_for_plot['Actual'],
            mode='lines+markers',
            name='Actual GDP',
            line=dict(color='blue'),
            visible=initial_visibility,
            showlegend=True
        ))
        # 2. Forecasted GDP Line
        fig.add_trace(go.Scatter(
            x=forecast_data_for_plot['ds'],
            y=forecast_data_for_plot['yhat'],
            mode='lines',
            name='Forecasted GDP',
            line=dict(color='red', dash='dash'),
            visible=initial_visibility,
            showlegend=True
        ))
        # 3. Forecast Upper Confidence Interval
        fig.add_trace(go.Scatter(
            x=forecast_data_for_plot['ds'],
            y=forecast_data_for_plot['yhat_upper'],
            fill=None,
            mode='lines',
            line_color='rgba(255,0,0,0)',
            showlegend=False,
            visible=initial_visibility
        ))
        # 4. Forecast Lower Confidence Interval (filled to upper)
        fig.add_trace(go.Scatter(
            x=forecast_data_for_plot['ds'],
            y=forecast_data_for_plot['yhat_lower'],
            fill='tonexty',
            mode='lines',
            line_color='rgba(255,0,0,0)',
            fillcolor='rgba(255,0,0,0.2)',
            name='Confidence Interval',
            visible=initial_visibility,
            showlegend=True
        ))

    # Create dropdown menu buttons
    buttons = []
    for i, region in enumerate(sorted_unique_regions):
        button_visibility = [False] * (len(sorted_unique_regions) * total_traces_per_region)
        button_visibility[i * total_traces_per_region] = True
        button_visibility[i * total_traces_per_region + 1] = True
        button_visibility[i * total_traces_per_region + 2] = True
        button_visibility[i * total_traces_per_region + 3] = True

        buttons.append(dict(
            label=region,
            method="update",
            args=[{"visible": button_visibility},
                  {"title": f"Regional GDP Forecast for {region}"}]
        ))

    if first_region:
        fig.update_layout(
            updatemenus=[
                go.layout.Updatemenu(
                    type="dropdown",
                    direction="down",
                    active=0,
                    x=0.0,
                    xanchor="left",
                    y=1.15,
                    yanchor="top",
                    buttons=buttons,
                    # Added font property to updatemenu if text is very long
                    font=dict(size=12), # You can experiment with font size
                    pad={"r": 10, "t": 10}
                )
            ],
            title_text=f"Regional GDP Forecast for {first_region}",
            yaxis_title="GDP Value",
            xaxis_title="Year",
            hovermode="x unified",
            height=600,
            # *** ADDED A FIXED WIDTH TO GIVE PLOTLY MORE SPACE TO LAYOUT COMPONENTS ***
            width=1200, # Increased width to provide more horizontal room
            showlegend=True,
            # Increased top and significantly increased right margin
            margin=dict(l=50, r=250, t=180, b=50), # Increased 'r' (right) margin to accommodate external legend
            # Position the legend outside the plot area, to the right
            legend=dict(
                x=1.05, # Position legend to the right of the plot area (values > 1)
                y=1,    # Position at the top (relative to the plot area)
                xanchor='left', # Anchor the left side of the legend box to x
                yanchor='top',  # Anchor the top side of the legend box to y
                bgcolor='rgba(255,255,255,0.7)',
                bordercolor='rgba(0,0,0,0.2)',
                borderwidth=1
            )
        )
    else:
        fig.update_layout(
            title_text="No sufficient regional GDP data to generate forecasts. Please check the 'Value Counts' above.",
            xaxis_title="Year",
            yaxis_title="GDP Value",
            height=600,
            showlegend=False
        )

    fig.show()

    # Save the chart as an HTML file, loading Plotly.js from CDN
    fig.write_html("Forecasted Regional GDP.html", include_plotlyjs='cdn', full_html=True)
    print("\nInteractive chart saved as 'Forecasted Regional GDP.html'")
    print("If the issue persists, please try opening the HTML file in a different browser (e.g., Chrome, Firefox).")
    print("If embedding on a web page, ensure no conflicting CSS is affecting the chart's rendering.")


except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please ensure it's uploaded to /content/.")
    print("If you just restarted the Colab runtime, you might need to re-upload the file.")
    print("You can upload it using: `from google.colab import files; uploaded = files.upload()`")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Attempting to load cleaned data from: /content/cleaned_data.csv
Error: The file '/content/cleaned_data.csv' was not found. Please ensure it's uploaded to /content/.
If you just restarted the Colab runtime, you might need to re-upload the file.
You can upload it using: `from google.colab import files; uploaded = files.upload()`


In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.express as px # Import plotly.express for interactive plotting
import os # Import os to check file extension

# --- Configuration ---
FILE_PATH = '/content/cleaned_data.xlsx' # Changed to .xlsx
TIME_COLUMN = 'Start_Year'
REGION_COLUMN = 'Region'
VALUE_COLUMN = 'Value'
METRIC_COLUMN = 'Metric'

TARGET_METRIC_NAME = 'Growth Rate'

# Set to 3 to ensure forecasts for 2023, 2024, and 2025 are generated
FORECAST_YEARS = 3
# --- End Configuration ---

def load_and_prepare_data(file_path, time_col, region_col, value_col, metric_col, target_metric_name):
    """
    Loads the data and performs initial preparation,
    including filtering by the specified target metric (Growth rate).
    Handles both CSV and Excel file types based on extension.
    """
    try:
        # Determine file type based on extension
        file_extension = os.path.splitext(file_path)[1].lower()
        if file_extension == '.csv':
            df = pd.read_csv(file_path)
            print(f"Loading data from CSV: {file_path}")
        elif file_extension == '.xlsx':
            df = pd.read_excel(file_path)
            print(f"Loading data from Excel: {file_path}")
        else:
            print(f"Error: Unsupported file type '{file_extension}'. Please provide a .csv or .xlsx file.")
            return pd.DataFrame()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found. Please ensure the file exists and the path is correct.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")
        return pd.DataFrame()

    # Clean column names (remove leading/trailing whitespace)
    df.columns = df.columns.str.strip()

    # Ensure essential columns exist after stripping
    required_cols = [time_col, region_col, value_col, metric_col]
    if not all(col in df.columns for col in required_cols):
        print(f"Error: One or more required columns ({', '.join(required_cols)}) not found in the DataFrame.")
        print(f"Available columns: {df.columns.tolist()}")
        return pd.DataFrame()

    # Filter data to include only the specified target metric (e.g., 'Growth Rate')
    df_target_metric_only = df[df[metric_col] == target_metric_name].copy()

    if df_target_metric_only.empty:
        print(f"Warning: No data found for '{target_metric_name}' in the '{metric_col}' column.")
        print(f"Available metrics: {df[metric_col].unique().tolist()}")
        return pd.DataFrame()

    # Convert time column to numeric, then to datetime for Prophet
    df_target_metric_only[time_col] = pd.to_numeric(df_target_metric_only[time_col], errors='coerce')
    # Drop rows where essential columns have NaN values after conversion
    df_target_metric_only = df_target_metric_only.dropna(subset=[time_col, value_col, region_col]).copy()

    # Prophet requires 'ds' (datetime) and 'y' (numeric) columns
    df_target_metric_only['ds'] = pd.to_datetime(df_target_metric_only[time_col], format='%Y')
    df_target_metric_only['y'] = pd.to_numeric(df_target_metric_only[value_col], errors='coerce') # Ensure value column is numeric

    return df_target_metric_only

def forecast_prophet_regional_metric(data, region_col, forecast_years):
    """
    Forecasts a specific metric (e.g., Growth Rate) for each region using Facebook Prophet.
    """
    regional_forecasts = {}
    regions = data[region_col].unique()

    for region in regions:
        region_df = data[data[region_col] == region].copy()

        if len(region_df) < 2:
            print(f"Skipping forecasting for region '{region}': Not enough historical data (need at least 2 points).")
            continue

        prophet_df = region_df[['ds', 'y']].rename(columns={'ds': 'ds', 'y': 'y'})

        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False,
            seasonality_mode='additive'
        )
        model.fit(prophet_df)

        future = model.make_future_dataframe(periods=forecast_years, freq='Y')
        forecast = model.predict(future)

        last_historical_year = prophet_df['ds'].max().year
        # Filter for future forecasts only
        future_forecast = forecast[forecast['ds'].dt.year > last_historical_year].copy()

        if future_forecast.empty:
            print(f"Warning: No future forecast generated for region '{region}'. Check data or forecast_years.")
            continue

        regional_forecasts[region] = {
            'model': model,
            'forecast_df': future_forecast,
            'historical_df': region_df
        }
    return regional_forecasts

# --- Main Execution Block ---
if __name__ == "__main__":
    # Load and filter data for 'Growth Rate' metric
    df_growth_rate_filtered = load_and_prepare_data(
        FILE_PATH, TIME_COLUMN, REGION_COLUMN, VALUE_COLUMN,
        METRIC_COLUMN, TARGET_METRIC_NAME
    )

    if df_growth_rate_filtered.empty:
        print("Data preparation failed or no 'Growth Rate' data found after filtering. Exiting.")
    else:
        print(f"\nAggregating '{TARGET_METRIC_NAME}' by region and year (averaging if multiple entries per year/region)...")
        # Group by Start_Year and Region, then calculate the mean of 'Value'
        df_regional_growth_rate = df_growth_rate_filtered.groupby([TIME_COLUMN, REGION_COLUMN]).agg(
            Avg_Growth_Rate=(VALUE_COLUMN, 'mean'),
            ds=('ds', 'first') # Keep the 'ds' column for Prophet
        ).reset_index()
        df_regional_growth_rate['y'] = df_regional_growth_rate['Avg_Growth_Rate']


        print(f"\nStarting regional '{TARGET_METRIC_NAME}' forecasting...")
        regional_forecasts = forecast_prophet_regional_metric(df_regional_growth_rate, REGION_COLUMN, FORECAST_YEARS)

        # --- Calculate Projected Average Growth Rate and Rank Regions ---
        ranked_regions_data = []
        target_years = [2024, 2025] # Years for calculating average projected growth

        print(f"\nCalculating average projected '{TARGET_METRIC_NAME}' for {target_years[0]}-{target_years[1]} and ranking regions...")
        for region, data in regional_forecasts.items():
            if 'forecast_df' in data and not data['forecast_df'].empty:
                forecast_for_target_years = data['forecast_df'][data['forecast_df']['ds'].dt.year.isin(target_years)]

                if not forecast_for_target_years.empty:
                    average_projected_growth_rate = forecast_for_target_years['yhat'].mean()
                    ranked_regions_data.append({
                        'Region': region,
                        f'Average_Projected_Growth_Rate_{target_years[0]}_{target_years[1]} (%)': average_projected_growth_rate
                    })
                else:
                    print(f"Skipping ranking for region '{region}': No forecast data found specifically for {target_years[0]} and {target_years[1]} in the filtered forecast_df.")

        # Create DataFrame for ranking
        if ranked_regions_data:
            df_ranked_regions = pd.DataFrame(ranked_regions_data)
            # Sort by average projected growth rate in descending order (highest growth first)
            df_ranked_regions = df_ranked_regions.sort_values(
                by=f'Average_Projected_Growth_Rate_{target_years[0]}_{target_years[1]} (%)',
                ascending=False
            ).reset_index(drop=True)
            df_ranked_regions.index += 1 # Adjust index to start ranking from 1

            # Define the column name for the growth rate
            growth_rate_col_name = f'Average_Projected_Growth_Rate_{target_years[0]}_{target_years[1]} (%)'

            # --- Plotly Chart Generation ---
            fig = px.bar(
                df_ranked_regions,
                x=growth_rate_col_name,
                y='Region',
                orientation='h', # Horizontal bar chart for better readability of long region names
                title=f'Future Leaders: Regions by Average Projected Growth Rate ({target_years[0]}-{target_years[1]})',
                labels={
                    growth_rate_col_name: 'Average Projected Growth Rate (%)',
                    'Region': 'Region'
                },
                color=growth_rate_col_name, # Color bars by growth rate
                color_continuous_scale=px.colors.sequential.Viridis # Use a sequential color scale
            )

            # Ensure the y-axis (Regions) is sorted by the growth rate
            fig.update_layout(yaxis={'categoryorder':'total ascending'})

            # Save the interactive chart to an HTML file
            chart_html_filename = f'Interactive_Regional_Projected_{TARGET_METRIC_NAME.replace(" ", "_")}_Ranking_{target_years[0]}_{target_years[1]}.html'
            fig.write_html(chart_html_filename)
            print(f"\nInteractive chart saved to '{chart_html_filename}'")

        else:
            print("No sufficient forecast data available to rank regions based on projected growth rates.")


INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/3dpwg5rm.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/lalwh7gb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=33296', 'data', 'file=/tmp/tmp33v_ubrc/3dpwg5rm.json', 'init=/tmp/tmp33v_ubrc/lalwh7gb.json', 'output', 'file=/tmp/tmp33v_ubrc/prophet_modelefz7rnri/prophet_model-20250705074657.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:46:57 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:46:57 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

Loading data from Excel: /content/cleaned_data.xlsx

Aggregating 'Growth Rate' by region and year (averaging if multiple entries per year/region)...

Starting regional 'Growth Rate' forecasting...


/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/etsm3ojn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/i0rugd6i.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=22099', 'data', 'file=/tmp/tmp33v_ubrc/etsm3ojn.json', 'init=/tmp/tmp33v_ubrc/i0rugd6i.json', 'output', 'file=/tmp/tmp33v_ubrc/prophet_modelvyw1jflj/prophet_model-20250705074657.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:46:57 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:46:57 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processin


Calculating average projected 'Growth Rate' for 2024-2025 and ranking regions...

Interactive chart saved to 'Interactive_Regional_Projected_Growth_Rate_Ranking_2024_2025.html'


In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.express as px

# --- Configuration ---
FILE_PATH = '/content/cleaned_data.xlsx'  # Replace with your actual path
TIME_COLUMN = 'Start_Year'
REGION_COLUMN = 'Region'
VALUE_COLUMN = 'Value'
METRIC_COLUMN = 'Metric'
TARGET_METRIC = 'GDP'
FORECAST_YEARS = 3

# Load and prepare GDP data (not growth rate)
def load_gdp_data(file_path):
    df = pd.read_excel(file_path)
    df.columns = df.columns.str.strip()
    df = df[df[METRIC_COLUMN] == TARGET_METRIC].copy()
    df[TIME_COLUMN] = pd.to_numeric(df[TIME_COLUMN], errors='coerce')
    df = df.dropna(subset=[TIME_COLUMN, VALUE_COLUMN, REGION_COLUMN])

    # Aggregate GDP across all regions per year
    df_total = df.groupby(TIME_COLUMN)[VALUE_COLUMN].sum().reset_index()
    df_total['ds'] = pd.to_datetime(df_total[TIME_COLUMN], format='%Y')
    df_total['y'] = df_total[VALUE_COLUMN]
    return df_total[['ds', 'y']]

# Forecast total GDP using Prophet
def forecast_total_gdp(df_total):
    model = Prophet(yearly_seasonality=True)
    model.fit(df_total)
    future = model.make_future_dataframe(periods=FORECAST_YEARS, freq='Y')
    forecast = model.predict(future)

    return forecast[['ds', 'yhat']]

# Combine historical and forecast data
def combine_actual_and_forecast(df_actual, df_forecast):
    df_actual = df_actual.copy()
    df_actual['Type'] = 'Historical'
    df_actual = df_actual.rename(columns={'y': 'GDP'})

    df_forecast = df_forecast.copy()
    df_forecast = df_forecast[df_forecast['ds'] > df_actual['ds'].max()]
    df_forecast['Type'] = 'Forecast'
    df_forecast = df_forecast.rename(columns={'yhat': 'GDP'})

    combined = pd.concat([df_actual, df_forecast])
    combined['Year'] = combined['ds'].dt.year
    return combined

# Plot using Plotly
def plot_gdp_line(combined_df):
    fig = px.line(
        combined_df,
        x='Year',
        y='GDP',
        color='Type',
        markers=True,
        labels={'GDP': 'Total GDP (PHP)', 'Year': 'Year'},
        title='Philippine Total GDP (2018–2025): Historical & Forecasted'
    )

    fig.update_traces(mode='lines+markers')
    fig.update_layout(template='plotly_white')
    fig.write_html('Total_GDP_Forecast_Line.html')
    fig.show()

# Run all steps
df_total = load_gdp_data(FILE_PATH)
df_forecast = forecast_total_gdp(df_total)
combined_df = combine_actual_and_forecast(df_total, df_forecast)
plot_gdp_line(combined_df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/anmfu9aj.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/2d7dcdjm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=12400', 'data', 'file=/tmp/tmp33v_ubrc/anmfu9aj.json', 'init=/tmp/tmp33v_ubrc/2d7dcdjm.json', 'output', 'file=/tmp/tmp33v_ubrc/prophet_modelu17e3li_/prophet_model-20250705075502.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:55:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:55:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:c

In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go # Changed to graph_objects for more control
import plotly.express as px # Still useful for color palettes
import os # Import os to check file extension

# === CONFIGURATION ===
FILE_PATH = '/content/cleaned_data.xlsx'
TIME_COLUMN = 'Start_Year'
INDUSTRY_COLUMN = 'Industry'
VALUE_COLUMN = 'Value'
METRIC_COLUMN = 'Metric'
TARGET_METRIC = 'GDP'  # Or whatever is used for raw GDP in your dataset
FORECAST_YEARS = 3 # Set to 3 to ensure forecasts for 2023, 2024, and 2025 are generated
# =====================

try:
    # STEP 1: Load and preprocess
    print(f"Attempting to load data from: {FILE_PATH}")
    # Determine file type based on extension
    file_extension = os.path.splitext(FILE_PATH)[1].lower()
    if file_extension == '.csv':
        df = pd.read_csv(FILE_PATH)
        print(f"Loading data from CSV: {FILE_PATH}")
    elif file_extension == '.xlsx':
        df = pd.read_excel(FILE_PATH)
        print(f"Loading data from Excel: {FILE_PATH}")
    else:
        raise ValueError(f"Unsupported file type '{file_extension}'. Please provide a .csv or .xlsx file.")

    df.columns = df.columns.str.strip()
    print(f"\nSuccessfully loaded data from: '{FILE_PATH}'")

    # Filter to total GDP values only
    df_gdp = df[df[METRIC_COLUMN] == TARGET_METRIC].copy()

    if df_gdp.empty:
        raise ValueError(f"No data found for metric '{TARGET_METRIC}'. Please check the 'Metric' column in your data.")

    # Group by year + industry (national level)
    # Summing values across all Location_Type and Location_Name to get national total per industry
    df_grouped = df_gdp.groupby([TIME_COLUMN, INDUSTRY_COLUMN]).agg(
        Total_GDP=(VALUE_COLUMN, 'sum')
    ).reset_index()

    # Prepare for Prophet: one model per industry
    df_grouped['ds'] = pd.to_datetime(df_grouped[TIME_COLUMN], format='%Y')
    df_grouped['y'] = df_grouped['Total_GDP']

    # Drop rows where 'y' is NaN, as Prophet will use historical data for training
    df_grouped.dropna(subset=['y'], inplace=True)

    # STEP 2: Forecast for each industry
    all_industry_historical = []
    all_industry_forecast = []

    unique_industries = sorted(df_grouped[INDUSTRY_COLUMN].unique().tolist())
    print(f"\nStarting GDP forecast for {len(unique_industries)} industries...")

    for industry in unique_industries:
        df_ind = df_grouped[df_grouped[INDUSTRY_COLUMN] == industry][['ds', 'y']].copy()

        # Ensure 'ds' column is sorted for Prophet
        df_ind = df_ind.sort_values('ds').reset_index(drop=True)

        if len(df_ind) < 2:
            print(f"Skipping forecasting for industry '{industry}': Not enough historical data (need at least 2 data points).")
            continue

        model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        model.fit(df_ind)

        future = model.make_future_dataframe(periods=FORECAST_YEARS, freq='Y')
        forecast = model.predict(future)

        # Prepare historical data for this industry
        hist_df = df_ind.copy()
        hist_df['Industry'] = industry
        hist_df.rename(columns={'y': 'Actual_GDP'}, inplace=True) # Rename 'y' to 'Actual_GDP' for clarity
        all_industry_historical.append(hist_df)

        # Prepare forecast data for this industry
        forecast_df = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
        forecast_df['Industry'] = industry
        all_industry_forecast.append(forecast_df)

    # Combine all historical and forecast dataframes
    df_historical_combined = pd.concat(all_industry_historical, ignore_index=True) if all_industry_historical else pd.DataFrame()
    df_forecast_combined = pd.concat(all_industry_forecast, ignore_index=True) if all_industry_forecast else pd.DataFrame()

    # Limit to 2025 for plotting (as per original code's intention for forecast_years=3)
    df_historical_combined = df_historical_combined[df_historical_combined['ds'].dt.year <= 2025]
    df_forecast_combined = df_forecast_combined[df_forecast_combined['ds'].dt.year <= 2025]

    if df_historical_combined.empty and df_forecast_combined.empty:
        raise ValueError("No data available to generate the chart after processing. Please check your input data and filters.")

    # STEP 3: Plotting with plotly.graph_objects for better control over historical/forecast/CI
    fig = go.Figure()

    # Get unique industries for consistent coloring
    # Ensure all unique industries from both historical and forecast data are covered
    all_industries_for_colors = sorted(list(set(df_historical_combined['Industry'].unique().tolist() + df_forecast_combined['Industry'].unique().tolist())))

    # Create a color map for industries
    colors = px.colors.qualitative.Plotly # Using Plotly's default qualitative palette
    color_map = {industry: colors[i % len(colors)] for i, industry in enumerate(all_industries_for_colors)}

    for industry in all_industries_for_colors:
        hist_data = df_historical_combined[df_historical_combined['Industry'] == industry].sort_values('ds')
        forecast_data = df_forecast_combined[df_forecast_combined['Industry'] == industry].sort_values('ds')

        # Plot Historical Data (Actual GDP)
        if not hist_data.empty:
            fig.add_trace(go.Scatter(
                x=hist_data['ds'],
                y=hist_data['Actual_GDP'],
                mode='lines+markers',
                name=f'{industry} (Actual)',
                legendgroup=industry, # Group traces for legend
                line=dict(color=color_map[industry]),
                showlegend=True
            ))

        # Plot Forecasted Data (yhat) and Confidence Interval
        if not forecast_data.empty:
            last_hist_year_data = hist_data[hist_data['ds'] == hist_data['ds'].max()] if not hist_data.empty else pd.DataFrame()

            # Combine last historical point with first forecast point to ensure continuous line
            # This helps in connecting the historical line to the forecast line smoothly
            combined_forecast_x = forecast_data['ds'].tolist()
            combined_forecast_y = forecast_data['yhat'].tolist()
            combined_forecast_y_lower = forecast_data['yhat_lower'].tolist()
            combined_forecast_y_upper = forecast_data['yhat_upper'].tolist()

            if not last_hist_year_data.empty:
                last_hist_ds = last_hist_year_data['ds'].iloc[0]
                last_hist_y = last_hist_year_data['Actual_GDP'].iloc[0]

                # Find the forecast point that corresponds to the last historical year or the year immediately after
                # This ensures the forecast line starts correctly relative to historical data
                forecast_start_point = forecast_data[forecast_data['ds'] == last_hist_ds]
                if forecast_start_point.empty:
                    forecast_start_point = forecast_data[forecast_data['ds'].dt.year == (last_hist_ds.year + 1)]

                if not forecast_start_point.empty:
                    # Add the last historical point to the beginning of the forecast line data
                    combined_forecast_x.insert(0, last_hist_ds)
                    combined_forecast_y.insert(0, last_hist_y)
                    combined_forecast_y_lower.insert(0, forecast_start_point['yhat_lower'].iloc[0])
                    combined_forecast_y_upper.insert(0, forecast_start_point['yhat_upper'].iloc[0])

            fig.add_trace(go.Scatter(
                x=combined_forecast_x,
                y=combined_forecast_y,
                mode='lines',
                name=f'{industry} (Forecast)',
                legendgroup=industry,
                line=dict(color=color_map[industry], dash='dash'),
                showlegend=True
            ))

            # Confidence Interval
            # Use the same combined_forecast_x for CI to align with the forecast line
            fig.add_trace(go.Scatter(
                x=combined_forecast_x + combined_forecast_x[::-1], # X-values for upper and lower bounds
                y=combined_forecast_y_upper + combined_forecast_y_lower[::-1], # Y-values for upper and then lower reversed
                fill='toself',
                fillcolor=f'rgba({int(color_map[industry][1:3], 16)},{int(color_map[industry][3:5], 16)},{int(color_map[industry][5:7], 16)},0.2)', # Use RGBA for transparency
                line=dict(width=0),
                hoverinfo='skip', # Don't show hover for the fill area
                showlegend=False,
                legendgroup=industry,
                name=f'{industry} Confidence Interval'
            ))

    # Update layout for a clear and informative chart
    fig.update_layout(
        title_text='Philippine Industry GDP Projection (2018–2025) with Forecast and Confidence Intervals',
        xaxis_title='Year',
        yaxis_title='GDP (PHP Billions)', # Assuming 'Value' is in Billions
        hovermode='x unified',
        height=700,
        legend_title_text='Industry Sector',
        xaxis=dict(
            tickmode='array',
            # Dynamically set tickvals based on data range, ensuring 2025 is included
            tickvals=list(range(df_historical_combined['ds'].min().year if not df_historical_combined.empty else 2018, 2026)),
            tickformat="%Y",
            showgrid=True # Show gridlines for better readability
        ),
        yaxis=dict(
            showgrid=True # Show gridlines for better readability
        ),
        margin=dict(l=50, r=50, t=100, b=50), # Adjust margins
        legend=dict(
            orientation="h", # Horizontal legend
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            bgcolor='rgba(255,255,255,0.7)',
            bordercolor='rgba(0,0,0,0.2)',
            borderwidth=1
        )
    )

    output_filename = "National_Industry_GDP_Forecast.html"
    fig.write_html(output_filename)
    print(f"✅ Chart saved as {output_filename}")

except FileNotFoundError:
    print(f"Error: The file '{FILE_PATH}' was not found. Please ensure it's uploaded to /content/.")
    print("If you just restarted the Colab runtime, you might need to re-upload the file.")
    print("You can upload it using: `from google.colab import files; uploaded = files.upload()`")
except ValueError as ve:
    print(f"Data Error: {ve}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")



Attempting to load data from: /content/cleaned_data.xlsx


INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/dw9ncjh0.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/0hvcah34.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=89267', 'data', 'file=/tmp/tmp33v_ubrc/dw9ncjh0.json', 'init=/tmp/tmp33v_ubrc/0hvcah34.json', 'output', 'file=/tmp/tmp33v_ubrc/prophet_modeldsrpf5w6/prophet_model-20250705075725.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:57:25 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:57:25 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

Loading data from Excel: /content/cleaned_data.xlsx

Successfully loaded data from: '/content/cleaned_data.xlsx'

Starting GDP forecast for 16 industries...


/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/u2rhpt78.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/mio72zkf.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=38719', 'data', 'file=/tmp/tmp33v_ubrc/u2rhpt78.json', 'init=/tmp/tmp33v_ubrc/mio72zkf.json', 'output', 'file=/tmp/tmp33v_ubrc/prophet_model2n7phzvb/prophet_model-20250705075725.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:57:25 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:57:25 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processin

✅ Chart saved as National_Industry_GDP_Forecast.html


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import os

def calculate_province_gdp_share(file_path='/content/cleaned_data.xlsx'):
    try:
        # Check file extension to read either CSV or Excel
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        else:
            print(f"Error: Unsupported file type. Please provide a .csv or .xlsx file: {file_path}")
            return None
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

    required_columns = ['Region', 'Location_Name', 'Metric', 'End_Year', 'Value']
    if not all(col in df.columns for col in required_columns):
        print(f"Error: Missing columns: {[col for col in required_columns if col not in df.columns]}")
        return None

    # Standardize Region Name: Replace "MIMAROPA REGION" with "Region IV-B (MIMAROPA)"
    if 'Region' in df.columns:
        df['Region'] = df['Region'].replace('MIMAROPA REGION', 'Region IV-B (MIMAROPA)')

    df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
    df['End_Year'] = pd.to_numeric(df['End_Year'], errors='coerce').astype(int)
    df.dropna(subset=['Value', 'End_Year'], inplace=True)
    df_filtered = df[(df['End_Year'] >= 2018) & (df['End_Year'] <= 2023)].copy()

    gdp_metrics = [m for m in df_filtered['Metric'].unique() if 'gdp' in str(m).lower()]
    if not gdp_metrics:
        print("No GDP metric found.")
        return None

    gdp_metric = gdp_metrics[0]
    df_gdp = df_filtered[df_filtered['Metric'] == gdp_metric]

    # Get unique region names to use for filtering
    region_names = df_gdp['Region'].unique()

    # Aggregate GDP by Region, Location_Name, and End_Year
    # IMPORTANT FIX: Exclude Location_Name entries that are identical to Region names
    df_gdp_provinces = df_gdp[~df_gdp['Location_Name'].isin(region_names)].copy()

    df_gdp_agg = df_gdp_provinces.groupby(['Region', 'Location_Name', 'End_Year'])['Value'].sum().reset_index()
    df_gdp_agg.rename(columns={'Value': 'Location_GDP'}, inplace=True)

    # Calculate Total Regional GDP for each Region and Year
    regional_totals = df_gdp.groupby(['Region', 'End_Year'])['Value'].sum().reset_index()
    regional_totals.rename(columns={'Value': 'Regional_Total_GDP'}, inplace=True)

    df_merged = pd.merge(df_gdp_agg, regional_totals, on=['Region', 'End_Year'], how='left')
    df_merged['Share_Within_Region'] = df_merged.apply(
        lambda row: (row['Location_GDP'] / row['Regional_Total_GDP']) * 100 if row['Regional_Total_GDP'] and row['Regional_Total_GDP'] != 0 else 0,
        axis=1
    )

    return df_merged

def plot_interactive_pie_with_region_and_year(df, output_html='gdp_share_by_region_year.html'):
    if df is None or df.empty:
        print("No data to plot for the chart.")
        return [], None

    regions = sorted(df['Region'].unique())
    years = sorted(df['End_Year'].unique())

    fig = go.Figure()

    trace_map = {}
    current_trace_index = 0

    for year in years:
        for region in regions:
            subset = df[(df['Region'] == region) & (df['End_Year'] == year)]

            if subset.empty or subset['Share_Within_Region'].sum() == 0:
                print(f"Warning: No valid data for {region} - {year}. Skipping trace for this combination.")
                fig.add_trace(go.Pie(
                    labels=[],
                    values=[],
                    name=f"{region} - {year}",
                    visible=False,
                    hovertemplate="No data available<extra></extra>",
                    textinfo="none"
                ))
            else:
                fig.add_trace(go.Pie(
                    labels=subset['Location_Name'],
                    values=subset['Share_Within_Region'],
                    name=f"{region} - {year}",
                    visible=False,
                    hovertemplate="<b>%{label}</b><br>Share: %{percent:.2f}%<extra></extra>",
                    textinfo="percent+label",
                    insidetextorientation="radial",
                    insidetextfont=dict(size=12)
                ))
            trace_map[(region, year)] = current_trace_index
            current_trace_index += 1

    if not fig.data:
        print("No valid traces could be created for the chart. Ensure your data has valid GDP shares for plotting.")
        return [], None

    default_region = regions[0] if regions else None
    default_year = years[0] if years else None

    initial_title = "GDP Share by Region and Year"

    found_default = False
    if default_region and default_year and (default_region, default_year) in trace_map:
        default_subset = df[(df['Region'] == default_region) & (df['End_Year'] == default_year)]
        if not default_subset.empty and default_subset['Share_Within_Region'].sum() > 0:
            fig.data[trace_map[(default_region, default_year)]].visible = True
            initial_title = f"GDP Share in {default_region} for {default_year}"
            found_default = True

    if not found_default:
        for r_temp in regions:
            for y_temp in years:
                if (r_temp, y_temp) in trace_map:
                    subset_check = df[(df['Region'] == r_temp) & (df['End_Year'] == y_temp)]
                    if not subset_check.empty and subset_check['Share_Within_Region'].sum() > 0:
                        fig.data[trace_map[(r_temp, y_temp)]].visible = True
                        default_region = r_temp
                        default_year = y_temp
                        initial_title = f"GDP Share in {default_region} for {default_year}"
                        found_default = True
                        break
            if found_default:
                break
        if not found_default:
            initial_title = "No sufficient data to display plot"
            print("No sufficient data found to display any default chart.")


    region_buttons = []
    for reg_btn in regions:
        visibility = [
            (trace_map_key[0] == reg_btn and trace_map_key[1] == default_year)
            for trace_map_key in trace_map.keys()
        ]
        region_buttons.append(dict(
            label=reg_btn,
            method="update",
            args=[{"visible": visibility},
                  {"title": f"GDP Share in {reg_btn} for {default_year}"}]
        ))

    year_buttons = []
    for yr_btn in years:
        visibility = [
            (trace_map_key[0] == default_region and trace_map_key[1] == yr_btn)
            for trace_map_key in trace_map.keys()
        ]
        year_buttons.append(dict(
            label=str(yr_btn),
            method="update",
            args=[{"visible": visibility},
                  {"title": f"GDP Share in {default_region} for {yr_btn}"}]
        ))

    fig.update_layout(
        title=dict(
            text=initial_title,
            y=0.95,
            x=0.05,
            xanchor='left',
            yanchor='top'
        ),
        margin=dict(t=180, b=50, l=100, r=200),
        height=650,
        width=1100,

        updatemenus=[
            {
                "buttons": region_buttons,
                "direction": "down",
                "x": 0.0,
                "xanchor": "left",
                "y": 1.05,
                "yanchor": "top",
                "showactive": True,
                "pad": {"r": 10, "t": 20, "b": 10},
                "active": regions.index(default_region) if default_region in regions else 0
            },
            {
                "buttons": year_buttons,
                "direction": "down",
                "x": 0.55, # Adjusted x for the second dropdown to prevent overlap
                "xanchor": "left",
                "y": 1.05,
                "yanchor": "top",
                "showactive": True,
                "pad": {"r": 10, "t": 20, "b": 10},
                "active": years.index(default_year) if default_year in years else 0
            }
        ],
        showlegend=True,
        legend=dict(
            x=1.02,
            y=1,
            xanchor='left',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='rgba(0,0,0,0.1)',
            borderwidth=1,
            itemclick="toggleothers",
            itemdoubleclick="toggle",
            font=dict(size=12)
        ),
        uniformtext_minsize=10,
        uniformtext_mode='hide',
    )

    pio.write_html(fig, output_html, auto_open=False)
    print(f"\n✅ Interactive chart saved as '{output_html}'.")

    return [fig], pio.to_html(fig, full_html=True, include_plotlyjs='cdn')


if __name__ == "__main__":
    input_file = '/content/cleaned_data.xlsx'
    output_excel = '/content/collab.xlsx'
    output_html = 'gdp_share_by_region_year.html'

    df_result = calculate_province_gdp_share(input_file)

    if df_result is not None and not df_result.empty:
        try:
            df_result.to_excel(output_excel, index=False)
            print(f"\n✅ GDP share data saved to '{output_excel}'.")
        except Exception as e:
            print(f"\nError saving data to Excel: {e}")
    else:
        print("\nNo GDP share data to save to Excel.")

    figures, chart_preview_html = plot_interactive_pie_with_region_and_year(df_result, output_html=output_html)

    if chart_preview_html:
        print("\n--- Chart Preview ---")
        print("Below is an HTML representation of the first generated interactive chart.")
        print("It should be rendered directly in the output/preview pane.")
        print("\n")
        print("")
        print("```html")
        print(chart_preview_html)
        print("```")
    else:
        print("\nNo chart preview available due to data issues or no charts being generated.")


✅ GDP share data saved to '/content/collab.xlsx'.

✅ Interactive chart saved as 'gdp_share_by_region_year.html'.

--- Chart Preview ---
Below is an HTML representation of the first generated interactive chart.
It should be rendered directly in the output/preview pane.



```html
<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: 'local'};</script>
        <script charset="utf-8" src="https://cdn.plot.ly/plotly-2.35.2.min.js"></script>                <div id="713628f7-daa8-43e5-8126-647dc86c1984" class="plotly-graph-div" style="height:650px; width:1100px;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("713628f7-daa8-43e5-8126-647dc86c1984")) {                    Plotly.newPlot(                        "713628f7-daa8-43e5-8126-647dc86c1984",     

In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os # Import os to check file extension

# Define the file path to the cleaned Excel file
file_path = r"/content/cleaned_data.xlsx"

try:
    print(f"Attempting to load cleaned data from: {file_path}")
    # Determine file type based on extension
    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension == '.csv':
        final_df = pd.read_csv(file_path)
        print(f"Loading data from CSV: {file_path}")
    elif file_extension == '.xlsx':
        final_df = pd.read_excel(file_path)
        print(f"Loading data from Excel: {file_path}")
    else:
        raise ValueError(f"Unsupported file type '{file_extension}'. Please provide a .csv or .xlsx file.")
    print(f"\nSuccessfully loaded cleaned data from: '{file_path}'")

    # Ensure correct data types, as reading from Excel might alter them
    final_df['Industry'] = final_df['Industry'].astype(str)
    final_df['Region'] = final_df['Region'].astype(str)
    final_df['Location_Type'] = final_df['Location_Type'].astype(str)
    final_df['Location_Name'] = final_df['Location_Name'].astype(str)
    final_df['Metric'] = final_df['Metric'].astype(str)
    final_df['Year_Range'] = final_df['Year_Range'].astype(str)
    # Use 'Int64' for integer columns that might contain NaN
    final_df['Start_Year'] = pd.to_numeric(final_df['Start_Year'], errors='coerce').astype('Int64')
    final_df['End_Year'] = pd.to_numeric(final_df['End_Year'], errors='coerce').astype('Int64')
    final_df['Value'] = pd.to_numeric(final_df['Value'], errors='coerce')

    # --- DEBUGGING: Inspect available values in critical columns (for my reference) ---
    print("\n--- Value Counts for 'Metric' in cleaned_data.xlsx ---")
    print(final_df['Metric'].value_counts().to_markdown())

    print("\n--- Value Counts for 'Location_Type' in cleaned_data.xlsx ---")
    print(final_df['Location_Type'].value_counts().to_markdown())

    print("\n--- Value Counts for 'Industry' in cleaned_data.xlsx (Top 20) ---")
    print(final_df['Industry'].value_counts().head(20).to_markdown())

    print("\n--- Value Counts for 'Region' in cleaned_data.xlsx (Top 20) ---")
    print(final_df['Region'].value_counts().head(20).to_markdown())
    print("\n--- Value Counts for 'Location_Name' in cleaned_data.xlsx (Top 20) ---")
    print(final_df['Location_Name'].value_counts().head(20).to_markdown())


    # --- FORECASTING NATIONWIDE GDP FOR THE PHILIPPINES ---

    # Filter for GDP data at the Country level for 'Philippines'
    # Assuming 'Philippines' is explicitly listed as a 'Location_Name' with 'Location_Type' as 'Country'
    gdp_philippines = final_df[
        (final_df['Metric'] == 'GDP') &
        (final_df['Location_Type'] == 'Country') &
        (final_df['Location_Name'] == 'Philippines')
    ].copy()

    # If no direct 'Country' level data for 'Philippines', try to aggregate from regions
    if gdp_philippines.empty:
        print("\nWarning: No direct 'Country' level GDP data found for 'Philippines'. Attempting to aggregate regional GDPs.")
        gdp_philippines = final_df[
            (final_df['Metric'] == 'GDP') &
            (final_df['Location_Type'] == 'Region') # Aggregate all regions to get a nationwide proxy
        ].copy()
        # Aggregate by summing GDP across all industries and regions for each Year
        gdp_data = gdp_philippines.groupby(['Start_Year'])['Value'].sum().reset_index()
        gdp_data['Location_Name'] = 'Philippines' # Label it as aggregated
    else:
        # Aggregate by summing GDP across all industries for the Philippines for each Year
        gdp_data = gdp_philippines.groupby(['Location_Name', 'Start_Year'])['Value'].sum().reset_index()

    # Convert 'Start_Year' to datetime for Prophet's 'ds' column
    gdp_data['ds'] = pd.to_datetime(gdp_data['Start_Year'], format='%Y')
    gdp_data['y'] = gdp_data['Value'] # Rename 'Value' to 'y' for Prophet

    # Drop rows where 'y' is NaN, as Prophet will use historical data for training
    gdp_data.dropna(subset=['y'], inplace=True)

    # Check if there's enough data for forecasting
    if gdp_data.empty or len(gdp_data) < 2:
        raise ValueError("Not enough historical data for Nationwide GDP of the Philippines to forecast. Need at least 2 data points.")

    print("\n--- Starting Nationwide GDP Forecast for the Philippines ---")

    # Prepare data for Prophet (single time series)
    philippines_df = gdp_data[['ds', 'y']].sort_values('ds')

    # Initialize and fit Prophet model
    m = Prophet(yearly_seasonality=True)
    m.fit(philippines_df)

    # Define the number of years to forecast into the future
    forecast_years = 5 # As per original script

    # Create future dataframe for forecasting
    future = m.make_future_dataframe(periods=forecast_years, freq='Y')
    forecast = m.predict(future)

    # Merge historical data with forecast for plotting
    forecast['Location_Name'] = gdp_data['Location_Name'].iloc[0] # Get the name (e.g., 'Philippines')
    merged_df = forecast.merge(philippines_df.rename(columns={'y': 'Actual'}), on='ds', how='left')
    merged_df = merged_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'Location_Name', 'Actual']]


    # --- Create Interactive Plotly Chart for Nationwide GDP ---

    fig = go.Figure()

    # 1. Actual GDP (Historical)
    fig.add_trace(go.Scatter(
        x=merged_df['ds'],
        y=merged_df['Actual'],
        mode='lines+markers',
        name='Actual GDP',
        line=dict(color='blue'),
        showlegend=True
    ))
    # 2. Forecasted GDP Line
    fig.add_trace(go.Scatter(
        x=merged_df['ds'],
        y=merged_df['yhat'],
        mode='lines',
        name='Forecasted GDP',
        line=dict(color='red', dash='dash'),
        showlegend=True
    ))
    # 3. Forecast Upper Confidence Interval
    fig.add_trace(go.Scatter(
        x=merged_df['ds'],
        y=merged_df['yhat_upper'],
        fill=None,
        mode='lines',
        line_color='rgba(255,0,0,0)',
        showlegend=False
    ))
    # 4. Forecast Lower Confidence Interval (filled to upper)
    fig.add_trace(go.Scatter(
        x=merged_df['ds'],
        y=merged_df['yhat_lower'],
        fill='tonexty',
        mode='lines',
        line_color='rgba(255,0,0,0)',
        fillcolor='rgba(255,0,0,0.2)',
        name='Confidence Interval',
        showlegend=True
    ))

    # Update layout for a single nationwide chart
    chart_title = f"Nationwide GDP Forecast for {merged_df['Location_Name'].iloc[0]}"
    fig.update_layout(
        title_text=chart_title,
        yaxis_title="GDP Value",
        xaxis_title="Year",
        hovermode="x unified",
        height=600,
        showlegend=True,
        margin=dict(l=50, r=50, t=80, b=50), # Adjusted top margin
        legend=dict(
            x=0.99,
            y=0.99,
            xanchor='right',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.7)',
            bordercolor='rgba(0,0,0,0.2)',
            borderwidth=1
        )
    )

    fig.show()

    # Add this line to save the chart as an HTML file
    output_filename = "Forecasted Nationwide GDP Philippines.html"
    fig.write_html(output_filename)
    print(f"\nInteractive chart saved as '{output_filename}'")

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please ensure it's uploaded to /content/.")
    print("If you just restarted the Colab runtime, you might need to re-upload the file.")
    print("You can upload it using: `from google.colab import files; uploaded = files.upload()`")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Attempting to load cleaned data from: /content/cleaned_data.xlsx


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/4df6hzc_.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/xg2rsnva.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=25751', 'data', 'file=/tmp/tmp33v_ubrc/4df6hzc_.json', 'init=/tmp/tmp33v_ubrc/xg2rsnva.json', 'output', 'file=/tmp/tmp33v_ubrc/prophet_model89vx8wfm/prophet_model-20250705074951.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:49:51 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


Loading data from Excel: /content/cleaned_data.xlsx

Successfully loaded cleaned data from: '/content/cleaned_data.xlsx'

--- Value Counts for 'Metric' in cleaned_data.xlsx ---
| Metric        |   count |
|:--------------|--------:|
| GDP           |   12870 |
| Percent Share |   12870 |
| Growth Rate   |   10725 |

--- Value Counts for 'Location_Type' in cleaned_data.xlsx ---
| Location_Type   |   count |
|:----------------|--------:|
| Province/City   |   21505 |
| City            |    8704 |
| Region          |    4624 |
| Province        |    1632 |

--- Value Counts for 'Industry' in cleaned_data.xlsx (Top 20) ---
| Industry                                                             |   count |
|:---------------------------------------------------------------------|--------:|
| Other Services                                                       |    2295 |
| Agriculture, Forestry, and Fishing                                   |    2278 |
| Manufacturing                          

07:49:51 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.




Interactive chart saved as 'Forecasted Nationwide GDP Philippines.html'


In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
import plotly.express as px
import os

# === CONFIGURATION ===
FILE_PATH = '/content/cleaned_data.xlsx'
TIME_COLUMN = 'Start_Year'
INDUSTRY_COLUMN = 'Industry'
VALUE_COLUMN = 'Value'
METRIC_COLUMN = 'Metric'
TARGET_METRIC_NAME = 'Growth Rate' # Target metric for forecasting
FORECAST_YEARS = 3 # To cover 2023, 2024, 2025 if last historical year is 2022
# =====================

def load_and_prepare_data(file_path, time_col, industry_col, value_col, metric_col, target_metric):
    """
    Loads the data, preprocesses it, and filters for the target metric.
    Handles both CSV and Excel file types.
    """
    try:
        file_extension = os.path.splitext(file_path)[1].lower()
        if file_extension == '.csv':
            df = pd.read_csv(file_path)
            print(f"Loading data from CSV: {file_path}")
        elif file_extension == '.xlsx':
            df = pd.read_excel(file_path)
            print(f"Loading data from Excel: {file_path}")
        else:
            raise ValueError(f"Unsupported file type '{file_extension}'. Please provide a .csv or .xlsx file.")
        print(f"\nSuccessfully loaded data from: '{file_path}'")

        df.columns = df.columns.str.strip()

        # Filter to the target metric (e.g., 'Growth Rate')
        df_filtered = df[df[metric_col] == target_metric].copy()

        if df_filtered.empty:
            print(f"Warning: No data found for '{target_metric}' in the '{metric_col}' column.")
            print(f"Available metrics: {df[metric_col].unique().tolist()}")
            return pd.DataFrame()

        # Group by year and industry, taking the mean of the value (assuming growth rate might have duplicates)
        # This aggregates to a national level growth rate for each industry per year
        df_grouped = df_filtered.groupby([time_col, industry_col]).agg(
            Avg_Growth_Rate=(value_col, 'mean')
        ).reset_index()

        # Prepare for Prophet: 'ds' (datetime) and 'y' (numeric value)
        df_grouped['ds'] = pd.to_datetime(df_grouped[time_col], format='%Y')
        df_grouped['y'] = pd.to_numeric(df_grouped['Avg_Growth_Rate'], errors='coerce')

        # Drop rows with NaN in 'y' after conversion
        df_grouped.dropna(subset=['y'], inplace=True)

        return df_grouped

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found. Please ensure it's uploaded to /content/.")
        print("If you just restarted the Colab runtime, you might need to re-upload the file.")
        print("You can upload it using: `from google.colab import files; uploaded = files.upload()`")
        return pd.DataFrame()
    except ValueError as ve:
        print(f"Data Error: {ve}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An unexpected error occurred during data loading and preparation: {e}")
        return pd.DataFrame()

def forecast_industry_growth(data, industry_col, forecast_years):
    """
    Forecasts growth rate for each industry using Facebook Prophet.
    Returns combined historical and forecast dataframes.
    """
    all_industry_historical = []
    all_industry_forecast = []

    unique_industries = sorted(data[industry_col].unique().tolist())
    print(f"\nStarting '{TARGET_METRIC_NAME}' forecast for {len(unique_industries)} industries...")

    for industry in unique_industries:
        df_ind = data[data[industry_col] == industry][['ds', 'y']].copy()
        df_ind = df_ind.sort_values('ds').reset_index(drop=True)

        if len(df_ind) < 2:
            print(f"Skipping forecasting for industry '{industry}': Not enough historical data (need at least 2 data points).")
            continue

        model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        model.fit(df_ind)

        future = model.make_future_dataframe(periods=forecast_years, freq='Y')
        forecast = model.predict(future)

        # Historical data for plotting
        hist_df = df_ind.copy()
        hist_df['Industry'] = industry
        hist_df.rename(columns={'y': 'Actual_Growth_Rate'}, inplace=True)
        all_industry_historical.append(hist_df)

        # Forecast data for plotting (including confidence intervals)
        forecast_df = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
        forecast_df['Industry'] = industry
        all_industry_forecast.append(forecast_df)

    df_historical_combined = pd.concat(all_industry_historical, ignore_index=True) if all_industry_historical else pd.DataFrame()
    df_forecast_combined = pd.concat(all_industry_forecast, ignore_index=True) if all_industry_forecast else pd.DataFrame()

    return df_historical_combined, df_forecast_combined

def plot_industry_growth_bar_chart(df_forecast, target_metric_name, target_years):
    """
    Generates a bar chart showing the average projected growth rate for each industry
    for the specified target years.
    """
    summary_data = []
    for industry in df_forecast['Industry'].unique():
        industry_forecast = df_forecast[df_forecast['Industry'] == industry].copy()

        # Filter for the target forecast years (e.g., 2024, 2025)
        forecast_for_target_years = industry_forecast[industry_forecast['ds'].dt.year.isin(target_years)]

        if not forecast_for_target_years.empty:
            avg_projected_growth = forecast_for_target_years['yhat'].mean()
            summary_data.append({
                'Industry': industry,
                f'Average Projected {target_metric_name} ({target_years[0]}-{target_years[1]}) (%)': avg_projected_growth
            })

    if summary_data:
        df_summary = pd.DataFrame(summary_data)
        # Sort by average projected growth rate in descending order (highest growth first)
        growth_rate_col_name = f'Average Projected {target_metric_name} ({target_years[0]}-{target_years[1]}) (%)'
        df_summary = df_summary.sort_values(
            by=growth_rate_col_name,
            ascending=True # Ascending for horizontal bar chart to have largest at top
        ).reset_index(drop=True)

        fig = px.bar(
            df_summary,
            x=growth_rate_col_name,
            y='Industry',
            orientation='h', # Horizontal bar chart
            title=f'Industry Performance Forecast: Average Projected {target_metric_name} ({target_years[0]}-{target_years[1]})',
            labels={
                growth_rate_col_name: f'Average Projected {target_metric_name} (%)',
                'Industry': 'Industry Sector'
            },
            color=growth_rate_col_name, # Color bars by growth rate
            color_continuous_scale=px.colors.sequential.Viridis # Use a sequential color scale
        )

        fig.update_layout(
            yaxis={'categoryorder':'total ascending'}, # Ensure y-axis (Industries) is sorted by growth rate
            margin=dict(l=50, r=50, t=80, b=50), # Adjust margins
            height=700, # Set a reasonable height
            hovermode="y unified" # Hover shows info for the bar
        )

        output_filename = f"Industry_Performance_Forecast_Bar_Chart_{target_years[0]}_{target_years[1]}.html"
        fig.write_html(output_filename)
        print(f"✅ Bar chart saved as {output_filename}")
        return df_summary
    else:
        print(f"\nNo sufficient forecast data available to create the bar chart for {target_metric_name} for {target_years[0]}-{target_years[1]}.")
        return pd.DataFrame()


def summarize_forecast(df_forecast, target_years):
    """
    Summarizes the average projected growth rate for each industry for target years.
    """
    summary_data = []
    for industry in df_forecast['Industry'].unique():
        industry_forecast = df_forecast[df_forecast['Industry'] == industry].copy()

        # Filter for the target forecast years (e.g., 2024, 2025)
        forecast_for_target_years = industry_forecast[industry_forecast['ds'].dt.year.isin(target_years)]

        if not forecast_for_target_years.empty:
            avg_projected_growth = forecast_for_target_years['yhat'].mean()
            summary_data.append({
                'Industry': industry,
                f'Average Projected {TARGET_METRIC_NAME} ({target_years[0]}-{target_years[1]}) (%)': avg_projected_growth
            })

    if summary_data:
        df_summary = pd.DataFrame(summary_data)
        df_summary = df_summary.sort_values(
            by=f'Average Projected {TARGET_METRIC_NAME} ({target_years[0]}-{target_years[1]}) (%)',
            ascending=False
        ).reset_index(drop=True)
        print(f"\n--- Average Projected {TARGET_METRIC_NAME} for 2024-2025 by Industry ---")
        print(df_summary.to_markdown(index=False))
        return df_summary
    else:
        print(f"\nNo sufficient forecast data available to summarize {TARGET_METRIC_NAME} for 2024-2025.")
        return pd.DataFrame()


# --- Main Execution Block ---
if __name__ == "__main__":
    df_growth_rate_data = load_and_prepare_data(
        FILE_PATH, TIME_COLUMN, INDUSTRY_COLUMN, VALUE_COLUMN, METRIC_COLUMN, TARGET_METRIC_NAME
    )

    if df_growth_rate_data.empty:
        print("Data preparation failed or no target metric data found. Exiting.")
    else:
        df_historical_growth, df_forecast_growth = forecast_industry_growth(
            df_growth_rate_data, INDUSTRY_COLUMN, FORECAST_YEARS
        )

        if not df_historical_growth.empty or not df_forecast_growth.empty:
            # Call the new bar chart plotting function
            target_summary_years = [2024, 2025]
            df_summary_for_plot = plot_industry_growth_bar_chart(df_forecast_growth, TARGET_METRIC_NAME, target_summary_years)

            # The summarization is now handled within the bar chart function, but we can still call it
            # if you need the markdown table printed separately.
            if not df_summary_for_plot.empty:
                # This call will just print the markdown table, as the plot is already generated
                summarize_forecast(df_forecast_growth, target_summary_years)
        else:
            print("No sufficient data to generate forecasts or plots for any industry.")



INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/w7nfb7_1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/zhknk0fe.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=97389', 'data', 'file=/tmp/tmp33v_ubrc/w7nfb7_1.json', 'init=/tmp/tmp33v_ubrc/zhknk0fe.json', 'output', 'file=/tmp/tmp33v_ubrc/prophet_modelta66khns/prophet_model-20250705081121.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
08:11:21 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:11:21 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

Loading data from Excel: /content/cleaned_data.xlsx

Successfully loaded data from: '/content/cleaned_data.xlsx'

Starting 'Growth Rate' forecast for 16 industries...


08:11:22 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.11/dist-packages/prophet/forecaster.py:1872: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/9x1u7uzb.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp33v_ubrc/zc_58_b3.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=4190', 'data', 'file=/tmp/tmp33v_ubrc/9x1u7uzb.json', 'init=/tmp/tmp33v_ubrc/zc_58_b3.json', 'output', 'file=/tmp/tmp33v_ubrc/prophet_modelc1inlbf6/prophet_model-20250705081122.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
08:11:22 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing

✅ Bar chart saved as Industry_Performance_Forecast_Bar_Chart_2024_2025.html

--- Average Projected Growth Rate for 2024-2025 by Industry ---
| Industry                                                             |   Average Projected Growth Rate (2024-2025) (%) |
|:---------------------------------------------------------------------|------------------------------------------------:|
| Mining and Quarrying                                                 |                                        39.0968  |
| Human Health and Social Work Activities                              |                                        30.8041  |
| Accommodation and Food Service Activities                            |                                        29.0553  |
| Transportation and Storage                                           |                                        23.7724  |
| Construction                                                         |                                        23.4562  

In [ ]:
import pandas as pd
import plotly.express as px
import json
import os
import requests # Import requests for loading GeoJSON

# === CONFIGURATION ===
FILE_PATH = '/content/cleaned_data.xlsx'
# Updated GeoJSON URL to point to the LOCAL FOLDER containing multiple regional JSON files
GEOJSON_URL = '/content/ph_region/' # IMPORTANT: Ensure this is the path to the FOLDER (e.g., /content/ph_region/)
# If you want to use the provincial map, change this to the folder containing provincial JSONs: '/content/ph_provinces/' (verify exact folder name)
# You can download region/province GeoJSONs from: https://github.com/macoymejia/geojsonph

TIME_COLUMN = 'Start_Year'
# Column in your Excel data that contains the geographic names (Region or Province)
LOCATION_NAME_COLUMN_FOR_MAP = 'Location_Name'
VALUE_COLUMN = 'Value'
METRIC_COLUMN = 'Metric'
TARGET_METRIC = 'GDP'
# Set the geographic level you want to map ('Region' or 'Province')
GEOGRAPHIC_LEVEL_TYPE = 'Region' # Currently set for regional map
# If you want to use the provincial map, change this to: 'Province'
# =====================

def load_data(file_path):
    """Loads data from CSV or Excel based on file extension."""
    try:
        file_extension = os.path.splitext(file_path)[1].lower()
        if file_extension == '.csv':
            df = pd.read_csv(file_path)
            print(f"Loading data from CSV: {file_path}")
        elif file_extension == '.xlsx':
            df = pd.read_excel(file_path)
            print(f"Loading data from Excel: {file_path}")
        else:
            raise ValueError(f"Unsupported file type '{file_extension}'. Please provide a .csv or .xlsx file.")
        df.columns = df.columns.str.strip() # Clean column names
        print(f"\nSuccessfully loaded data from: '{file_path}'")
        return df
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found. Please ensure it's uploaded to /content/.")
        print("If you just restarted the Colab runtime, you might need to re-upload the file.")
        print("You can upload it using: `from google.colab import files; uploaded = files.upload()`")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during data loading: {e}")
        return None

def load_geojson(path_or_url):
    """
    Loads GeoJSON data. If path_or_url is a directory, it combines all .json files within it.
    Otherwise, it treats it as a single file path or URL.
    """
    combined_features = []

    if os.path.isdir(path_or_url): # If it's a directory
        print(f"Loading GeoJSON files from directory: {path_or_url}")
        json_files = [f for f in os.listdir(path_or_url) if f.endswith('.json')]
        if not json_files:
            print(f"Error: No .json files found in the directory '{path_or_url}'.")
            return None

        for json_file in json_files:
            file_path = os.path.join(path_or_url, json_file)
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)
                    if 'features' in data:
                        combined_features.extend(data['features'])
                    elif data.get('type') == 'Feature': # If it's a single Feature object
                        combined_features.append(data)
                    else:
                        print(f"Warning: File '{json_file}' in '{path_or_url}' does not contain 'features' or is not a single 'Feature' object. Skipping.")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON from file '{json_file}': {e}. Skipping this file.")
            except Exception as e:
                print(f"An error occurred reading file '{json_file}': {e}. Skipping this file.")

        if not combined_features:
            print(f"Error: No valid GeoJSON features could be loaded from directory '{path_or_url}'.")
            return None

        # Construct a single GeoJSON FeatureCollection
        combined_geojson = {
            "type": "FeatureCollection",
            "features": combined_features
        }
        print(f"Successfully combined {len(combined_features)} features from directory '{path_or_url}'.")
        return combined_geojson

    else: # Assume it's a single file path or URL
        try:
            if path_or_url.startswith('/content/'): # Local file path
                with open(path_or_url, 'r') as f:
                    geojson_data = json.load(f)
                print(f"Successfully loaded local GeoJSON from: {path_or_url}")
            else: # URL
                response = requests.get(path_or_url)
                response.raise_for_status()
                geojson_data = response.json()
                print(f"Successfully loaded GeoJSON from URL: {path_or_url}")
            return geojson_data
        except FileNotFoundError:
            print(f"Error: Local GeoJSON file '{path_or_url}' not found. Please ensure it's uploaded to /content/.")
            return None
        except requests.exceptions.RequestException as e:
            print(f"Error loading GeoJSON from URL: {e}")
            print("Please check the URL or try providing a local GeoJSON file.")
            return None
        except json.JSONDecodeError as e:
            print(f"Error decoding GeoJSON: {e}. The file might not be a valid JSON.")
            return None
        except Exception as e:
            print(f"An unexpected error occurred during GeoJSON loading: {e}")
            return None

def prepare_gdp_data_for_map(df, time_col, location_name_col, value_col, metric_col, target_metric, geographic_level_type):
    """
    Filters and aggregates GDP data for mapping based on the specified geographic level.
    Ensures data is at the correct level and ready for merging.
    """
    # Filter for GDP metric and the specified geographic level (Region or Province)
    df_gdp_level = df[
        (df[METRIC_COLUMN] == TARGET_METRIC) &
        (df['Location_Type'] == geographic_level_type)
    ].copy()

    if df_gdp_level.empty:
        print(f"Warning: No {geographic_level_type} GDP data found for metric '{TARGET_METRIC}'.")
        print(f"Available metrics: {df[METRIC_COLUMN].unique().tolist()}")
        print(f"Available Location_Types for GDP: {df[df[METRIC_COLUMN] == TARGET_METRIC]['Location_Type'].unique().tolist()}")
        return pd.DataFrame()

    # Aggregate GDP by Location Name (Region/Province) and Year
    # Summing 'Value' to get total GDP for each location per year
    df_gdp_aggregated = df_gdp_level.groupby([time_col, location_name_col]).agg(
        Total_GDP=(value_col, 'sum')
    ).reset_index()

    # Ensure 'Total_GDP' is numeric
    df_gdp_aggregated['Total_GDP'] = pd.to_numeric(df_gdp_aggregated['Total_GDP'], errors='coerce')
    df_gdp_aggregated.dropna(subset=['Total_GDP'], inplace=True) # Drop rows with NaN GDP

    # Convert 'Start_Year' to string for animation_frame
    df_gdp_aggregated[time_col] = df_gdp_aggregated[time_col].astype(str)

    print(f"\nPrepared GDP data for mapping. Number of entries: {len(df_gdp_aggregated)}")
    print(f"Years available: {df_gdp_aggregated[time_col].unique().tolist()}")

    # --- DEBUGGING: Print unique location names from your data ---
    print(f"\nUnique '{location_name_col}' values in your data (for {geographic_level_type}s with GDP):")
    print(df_gdp_aggregated[location_name_col].unique().tolist())
    # --- END DEBUGGING ---

    return df_gdp_aggregated

def create_gdp_map(df_gdp, geojson_data, location_name_col, time_col, geographic_level_type):
    """
    Creates an interactive choropleth map of regional/provincial GDP.
    """
    if df_gdp.empty or geojson_data is None:
        print("Cannot create map: Missing GDP data or GeoJSON data.")
        return

    # Map GeoJSON feature properties to a unique ID for merging
    # For macoymejia/geojsonph, 'properties.name' usually holds the name
    # and 'id' is the unique feature ID.
    geojson_id_map = {feature['properties']['name']: feature['id'] for feature in geojson_data['features']}

    # --- DEBUGGING: Print unique names from GeoJSON ---
    print("\nUnique 'name' properties from GeoJSON features (from the loaded GeoJSON file):")
    print(list(geojson_id_map.keys()))
    # --- END DEBUGGING ---

    # Create a 'geojson_id' column in your DataFrame for merging
    # This assumes LOCATION_NAME_COLUMN_FOR_MAP in your data matches 'name' in GeoJSON
    df_gdp['geojson_id'] = df_gdp[location_name_col].map(geojson_id_map)

    # Filter out locations that don't have a match in the GeoJSON
    df_gdp_mapped = df_gdp.dropna(subset=['geojson_id']).copy()

    if df_gdp_mapped.empty:
        print(f"Warning: No matching {geographic_level_type}s found between your GDP data and the GeoJSON file.")
        print(f"Please check if {geographic_level_type} names in your data's '{location_name_col}' column match 'name' in the GeoJSON.")
        print(f"{geographic_level_type}s in your data: {df_gdp[location_name_col].unique().tolist()}")
        print(f"{geographic_level_type}s in GeoJSON (name property): {list(geojson_id_map.keys())}")
        return

    # Create the choropleth map
    fig = px.choropleth_mapbox(
        df_gdp_mapped,
        geojson=geojson_data,
        locations='geojson_id', # Column in df_gdp_mapped that contains the GeoJSON feature IDs
        color='Total_GDP', # Column to color the map by
        featureidkey="id", # Key in GeoJSON features to match 'locations'
        animation_frame=time_col, # Column for animation (years)
        color_continuous_scale="Viridis", # Color scale for GDP values
        range_color=(df_gdp_mapped['Total_GDP'].min(), df_gdp_mapped['Total_GDP'].max()), # Consistent color range
        mapbox_style="carto-positron", # Map style
        zoom=5, # Initial zoom level for Philippines
        center={"lat": 12.8797, "lon": 121.7740}, # Center of the Philippines
        opacity=0.7,
        labels={'Total_GDP': 'GDP (PHP Billions)', time_col: 'Year'},
        title=f'Philippine {geographic_level_type}al GDP by Year'
    )

    fig.update_layout(
        margin={"r":0,"t":50,"l":0,"b":0},
        height=700,
        coloraxis_colorbar=dict(
            title="GDP (PHP Billions)",
            thicknessmode="pixels", thickness=30,
            lenmode="pixels", len=300,
            yanchor="middle", y=0.5,
            xanchor="right", x=0.95
        )
    )

    # Save the interactive map to an HTML file
    output_filename = f"Philippine_{geographic_level_type}_GDP_Map.html"
    fig.write_html(output_filename)
    print(f"✅ Interactive map saved as {output_filename}")

# --- Main Execution Block ---
if __name__ == "__main__":
    # 1. Load your cleaned data
    df_cleaned = load_data(FILE_PATH)
    if df_cleaned is None:
        exit() # Exit if data loading failed

    # 2. Load GeoJSON data (from local file or URL)
    # This will now combine all JSON files in the specified directory
    geojson_data = load_geojson(GEOJSON_URL)
    if geojson_data is None:
        exit() # Exit if GeoJSON loading failed

    # 3. Prepare GDP data for mapping
    df_gdp_for_map = prepare_gdp_data_for_map(
        df_cleaned, TIME_COLUMN, LOCATION_NAME_COLUMN_FOR_MAP, VALUE_COLUMN, METRIC_COLUMN, TARGET_METRIC, GEOGRAPHIC_LEVEL_TYPE
    )
    if df_gdp_for_map.empty:
        print("No suitable GDP data found for mapping. Exiting.")
        exit()

    # 4. Create and save the interactive map
    create_gdp_map(df_gdp_for_map, geojson_data, LOCATION_NAME_COLUMN_FOR_MAP, TIME_COLUMN, GEOGRAPHIC_LEVEL_TYPE)


Loading data from Excel: /content/cleaned_data.xlsx

Successfully loaded data from: '/content/cleaned_data.xlsx'
Error: Local GeoJSON file '/content/ph_region/' not found. Please ensure it's uploaded to /content/.

Prepared GDP data for mapping. Number of entries: 102
Years available: ['2018', '2019', '2020', '2021', '2022', '2023']

Unique 'Location_Name' values in your data (for Regions with GDP):
['Bangsamoro Autonomous Region in Muslim Mindanao (BARMM)', 'Cordillera Administrative Region (CAR)', 'National Capital Region (NCR)', 'Region I (Ilocos Region)', 'Region II (Cagayan Valley)', 'Region III (Central Luzon)', 'Region IV-A (CALABARZON)', 'Region IV-B (MIMAROPA)', 'Region IX (Zamboanga Peninsula)', 'Region V (Bicol Region)', 'Region VI (Western Visayas)', 'Region VII (Central Visayas)', 'Region VIII (Eastern Visayas)', 'Region X (Northern Mindanao)', 'Region XI (Davao Region)', 'Region XII (SOCCSKSARGEN)', 'Region XIII (Caraga)']
Cannot create map: Missing GDP data or GeoJSON da